### Text generation

In [1]:
input = "Dynamic Programming" # Change the Input to generate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

# === Config ===
base_model = "mistralai/Mistral-7B-v0.1"
base = AutoModelForCausalLM.from_pretrained(base_model)
model = PeftModel.from_pretrained(base, "markteammate/Mistral_academic_style_tune")

# === Tokenizer === 
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

# === QLoRA quantization config ===
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# === Load base model with quantization ===
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
)

# === Load LoRA adapter from Hugging Face ===
model = PeftModel.from_pretrained(base_model, lora_repo)
model.eval()

# === Prompt ===
prompt = """### Instruction:
Write an academic paragraph given the title.

### Input: """ + input + """

### Response:
"""

# === Tokenize and generate ===
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.95,
        do_sample=True
    )

# === Decode ===
print(tokenizer.decode(output[0], skip_special_tokens=True))
